There are two datasets, containing company information. The first dataset has standardized publically available information obtained via web scrapping, the second dataset contains confidential internal company information. Thus, we have extracted only fields related to address from it. The task at hand is to match the data items between the two datasets as best as possible. 

In [1]:
import pandas as pd

# Read DataFrames
data2 = pd.read_csv('data2.csv')
data1 = pd.read_csv('data1.csv')

#Check for empty cells
print(data1.isnull().any())
print(data2.isnull().any())

Company.Name    False
Address          True
City             True
Postcode         True
dtype: bool
Address        False
Postal.Code     True
Location        True
dtype: bool


In [2]:
#Show DataFrames
data1.head()

,Company.Name,Address,City,Postcode
0,Carsten Helm,Ulmenstr. 8,Wismar,23966
1,Zirpel & Pautzsch Ingenieur Pa,Paditzer Str. 33,Altenburg,4600
2,Eberhard Zessin,Steingartenweg 12,Heidelberg,69118
3,Gerold Fuchs,Mühlweg 12,Dietingen,78661
4,Rudi Biedritzky,Zaisentalstr. 70/1,Reutlingen,72760


In [3]:
data2.head()

,Address,Postal.Code,Location
0,Alfons-Müller-Platz,50389,Wesseling
1,Hauptstr. 10B,66459,Kirkel
2,Edisonstr. 36,04435,Schkeuditz
3,Kutterstr. 3 / 26386 Wilhelmshaven-Rüstersiel,26386,Wilhelmshaven
4,Am neuen Markt 8,66877,Ramstein-Miesenbach


In [4]:
#Prepearing Data
def filter_data(data):
    
    #Replace "Str", 'Straße'
    strasse = ['str', 'straße']
    data['Address'] = data['Address'].astype('str')
    for k in data.index:
        
        s = data.ix[k, 'Address'].split()
        s = [x.replace('.','') for x in s]
        s  = [word for word in s if word.lower() not in strasse]
        s = ' '.join(s)
        data.ix[k, 'Address'] = s
    
    #Drop rows with empty cells 
    data.dropna(axis = 0, how = 'any', inplace = True)
 
    #Converts string to its upper case format (for similar case format)
    for col in data.columns:
        data[col] = data[col].str.upper()
    return data

In [5]:
data1 = filter_data(data1)
data2 = filter_data(data2)

C:\Users\SS\Anaconda\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [6]:
data1.head()

,Company.Name,Address,City,Postcode
0,CARSTEN HELM,ULMENSTR 8,WISMAR,23966
1,ZIRPEL & PAUTZSCH INGENIEUR PA,PADITZER 33,ALTENBURG,4600
2,EBERHARD ZESSIN,STEINGARTENWEG 12,HEIDELBERG,69118
3,GEROLD FUCHS,MÜHLWEG 12,DIETINGEN,78661
4,RUDI BIEDRITZKY,ZAISENTALSTR 70/1,REUTLINGEN,72760


In [7]:
data2.head()

,Address,Postal.Code,Location
0,ALFONS-MÜLLER-PLATZ,50389,WESSELING
1,HAUPTSTR 10B,66459,KIRKEL
2,EDISONSTR 36,04435,SCHKEUDITZ
3,KUTTERSTR 3 / 26386 WILHELMSHAVEN-RÜSTERSIEL,26386,WILHELMSHAVEN
4,AM NEUEN MARKT 8,66877,RAMSTEIN-MIESENBACH


In [8]:
#To determine the similarity of strings, we use the Levenshtein distance. Esspecialy Levenshtein.ratio = [0,1]

import Levenshtein
import statistics as st

ind = []
for k in data2.index:
    for j in data1.index:
        
        #Firstly compare Postcodes and Cities(Locations) from two DataFrames. 
        #Use Levenshtein.ratio = 0.7. Permissible for human's mistakes  
        
        if (Levenshtein.ratio(data2.ix[k, 'Postal.Code'], data1.ix[j, 'Postcode']) > 0.7) & \
        (Levenshtein.ratio(data2.ix[k, 'Location'], data1.ix[j, 'City']) > 0.7):
                
                #Then analyse addresses
                l = []
                
                #Divide into separate words for both DataFrames
                sentences = data2.ix[k, 'Address'].split(), data1.ix[j, 'Address'].split()
                
                """We exclude addresses where only one value (for example street or house number) 
                because such values are not suitable for comparison"""
                if (len(sentences[0]) <= 1) or (len(sentences[1]) <= 1):
                    continue
                
                for w1,w2 in zip(sentences[0],sentences[1]):
                    #Calculate mean Levenshtein ratio and choose items with ratio > 0.7
                    s = Levenshtein.ratio(w1,w2)
                    l.append(Levenshtein.ratio(w1,w2))
                    h = st.mean(l)
                if h > 0.7:
                    
                    #Thus we obtain relations between two Datasets 
                    print('Numbers of strings in 2nd and 1st Dataframes: ', k, j)
                    ind.append([j, k])
                    print('Addresses: ', sentences)
                    print('Postcodes: ', data2.ix[k, 'Postal.Code'], data1.ix[j, 'Postcode'])
                    print('Locations: ', data2.ix[k, 'Location'], data1.ix[j, 'City'])
                    print('Company name: ', data1.ix[j, 'Company.Name'])
                    print('---------------')

Numbers of strings in 2nd and 1st Dataframes:  11 4114
Addresses:  (['MELDORFER', '101'], ['MELDORFER', '101'])
Postcodes:  25746 25746
Locations:  HEIDE HEIDE
Company name:  NORDDEUTSCHE ELEKTROGROSSHANDEL
---------------
Numbers of strings in 2nd and 1st Dataframes:  19 2690
Addresses:  (['THEODOR-HEUSS-ALLEE', '20'], ['THEODOR-HEUSS-ALLEE', '21'])
Postcodes:  28215 28215
Locations:  BREMEN BREMEN
Company name:  GASSE-SCHUMACHER-SCHRAMM LANDS
---------------
Numbers of strings in 2nd and 1st Dataframes:  34 1766
Addresses:  (['WEIERSTR', '54'], ['WEIERSTR', '54'])
Postcodes:  53894 53894
Locations:  MECHERNICH MECHERNICH
Company name:  ELEKTRO-BRAUN MECHERNICH GMBH
---------------
Numbers of strings in 2nd and 1st Dataframes:  92 5396
Addresses:  (['SCHLOSSSTR', '13'], ['SCHLOSSSTR', '13'])
Postcodes:  04425 4425
Locations:  TAUCHA TAUCHA
Company name:  STADT TAUCHA
---------------
Numbers of strings in 2nd and 1st Dataframes:  107 2433
Addresses:  (['ALLEE', 'DER', 'KOSMONAUTEN', '2

## SUMMURY:

- We have relations from two tables.
- 6 of them are strong.
- In one (second) there is a mismatching of the house number (# 20 instead of # 21). It can be a human mistake, and maybe different addresses. To avoid this, we can put a more strong condition for the Levenshtein coefficient under address analyzing (for example > 0.8, 0.9). I left it as is to show the different assumption can be made.
- Also there are objects with similar addresses but in different cities

Plus:
- To solve this problem, we can use the geocoder. From the addresses we can get the coordinates and then compare them in the specified range (delta).